In [7]:
%load_ext autoreload
%autoreload 2

from datasets import Zone1
ds = Zone1('data/zone1_prior_load.parquet')
ds.data.head()

from validation import walkforward, Error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

model = RandomForestRegressor(n_estimators=70, max_samples=0.5, n_jobs=4)
(predictions, errors) = walkforward(model, ds.data, ds.data[ds.actual],
                                    ds.validation_start, ds.validation_end, Zone1.next_hour,
                                    ds.features)
overall_error = Error(ds.validation_data[ds.actual], np.concatenate(predictions))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: Found input variables with inconsistent numbers of samples: [8760, 8832]

In [2]:
overall_error

Error(mae=523.8949901141551, max=3336.975000000002, total=4589320.113400012)

In [6]:
cols = ds.data.columns.to_series()
cols = cols.drop([ds.actual, ds.mtlf])
(predictions, errors) = walkforward(model, ds.data, ds.data[ds.actual],
                                    ds.validation_start, ds.validation_end, Zone1.next_hour,
                                    cols)

KeyboardInterrupt: 